In [8]:
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical

# Define an image data generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255
)

# Define a function to preprocess an individual image
def preprocess_image(image):
    # Check the shape of the input image
    print(image.shape)

    # Convert the image to RGB if it has only one channel
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # Resize the image to a smaller size
    image = cv2.resize(image, (256, 256))

    # Normalize the pixel values to a common scale
    image = image / 255.

    return image

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model with a loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load the image data from files
train_datagen = ImageDataGenerator(
    # Data augmentation settings and normalization
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    'brain_tumor_dataset',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

# Preprocess the image data
x_train, y_train = next(train_generator)
x_train = np.array([preprocess_image(image) for image in x_train])

# Convert the target labels to binary matrices
y_train = to_categorical(y_train)

# Reshape the target labels
y_train = np.reshape(y_train, (len(y_train), 1))

# Define the temperature for defensive distillation
temperature = 10

# Perform defensive distillation
defensive_logits = model.predict(x_train)
soft_labels = np.exp(defensive_logits / temperature) / np.sum(np.exp(defensive_logits / temperature), axis=1, keepdims=True)

# Retrain the model with soft labels
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=10)


Found 253 images belonging to 2 classes.
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


ValueError: cannot reshape array of size 64 into shape (32,1)

In [ ]:
# Save the trained model to a file
model.save('preprocessingKerasModel1.h5')